In [1]:
!pip3 install flash-attention

In [2]:
import pandas as pd
data = pd.read_json('cladder.json')
data = data.dropna().reset_index(drop=True)

data = data.assign(step0 = lambda x: (x['reasoning']))
df = data['step0'].apply(lambda x: (x.get("step0")))
data['step0'] = df

data = data.assign(step1 = lambda x: (x['reasoning']))
df = data['step1'].apply(lambda x: (x.get("step1")))
data['step1'] = df

#step2) query type
data = data.assign(step2 = lambda x: (x['meta']))
df = data['step2'].apply(lambda x: (x.get("query_type")))
data['step2'] = df

#step3) formalize query
data = data.assign(step3 = lambda x: (x['reasoning']))
df = data['step3'].apply(lambda x: (x.get("step2")))
data['step3'] = df

#step4) extract all available data
data = data.assign(step4 = lambda x: (x['reasoning']))
df = data['step4'].apply(lambda x: (x.get("step4")))
data['step4'] = df

#step5) deduce estimand
data = data.assign(step5 = lambda x: (x['reasoning']))
df = data['step5'].apply(lambda x: (x.get("step3")))
data['step5'] = df

#step6)Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations
data = data.assign(step6 = lambda x: (x['reasoning']))
df = data['step6'].apply(lambda x: (x.get("step5")))
data['step6'] = df

#end) derive the final answer
data = data.assign(end = lambda x: (x['reasoning']))
df = data['end'].apply(lambda x: (x.get("end")))
data['end'] = df

data

,question_id,desc_id,given_info,question,answer,meta,reasoning,step0,step1,step2,step3,step4,step5,step6,end
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",nde,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) ...","\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y...",0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = ...,0.32 > 0
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",ate,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.26\nP(Y=1 | X=1) = 0.76,P(Y=1|X=1) - P(Y=1|X=0),0.76 - 0.26 = 0.50,0.50 > 0
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",marginal,P(Y),P(X=1) = 0.77\nP(Y=1 | X=0) = 0.26\nP(Y=1 | X=...,P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0),0.77*0.76 - 0.23*0.26 = 0.64,0.64 > 0
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",ate,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.20\nP(Y=1 | X=1) = 0.68,P(Y=1|X=1) - P(Y=1|X=0),0.68 - 0.20 = 0.49,0.49 > 0
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",nie,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.11\nP(Y=1 | X=0, V2=1) ...","\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X...",0.01 * (0.60 - 0.11)+ 0.61 * (0.92 - 0.46)= -0.29,-0.29 < 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=0} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 0 or 1,1
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=1} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 1 or 1,1
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is not glimx if zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=1} = 0 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 1 or 1,0
8530,31016,nonsense9-fork-det-counterfactual-model3843-sp...,We know that zuph and jyka causes glimx. We ob...,Would an individual is glimx

In [3]:
#create a list of real variable names and their respective representation e.g. [['X','wife'],['Y','husband']]
def list_representation(x):
    x = x.get('step0')
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    aux = []
    for i in range(len(x)):
        aux.append(x[i].split(" = "))
    return aux

#receives a list from list_representation and returns the prompt with it
def prompt_repres(x):
    x = x.get('step0')
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    list_rep = []
    for i in range(len(x)):
        list_rep.append(x[i].split(" = "))
    
    prompt = ""
    for i in range(len(list_rep)):
        prompt += "Use "+list_rep[i][0]+" to represent "+list_rep[i][1]+". "
    return prompt

def create_string(str_list):
    string = ""
    for i in range(str_list):
        string += str_list[i]
    return string

#exchange query_type abbreviation to the whole word
def expand(abb):
    if abb == 'nde':
        return 'natural direct effect'
    elif abb == 'ate':
        return 'average treatment effect'
    elif abb == 'marginal':
        return 'marginal probability'
    elif abb == 'nie':
        return 'natural indirect effect'
    elif abb == 'det-counterfactual':
        return 'normal counterfactual question'
    elif abb == 'ett':
        return 'average treatment effect on treated'
    elif abb == 'correlation':
        return 'conditional probability'
    elif abb == 'exp_away':
        return 'explaining away effect'
    elif abb == 'collider_bias':
        return 'collider bias'
    elif abb == 'backadj':
        return 'backdoor adjustment set'
    return 0
    

#data['formal_form'] = data.apply(lambda x: exchange(x['reasoning'], x['meta'],1), axis=1)
#data['graph'] = data.apply(lambda x: exchange(x['reasoning'], x['reasoning'],0), axis=1)
#data['info'] = data.apply(lambda x: exchange(x['reasoning'], x['info'],2), axis=1)
data['step2'] = data.apply(lambda x: expand(x['step2']), axis=1)

#data

In [4]:
data.iloc[4]

question_id                                                   21
desc_id                      alarm-mediation-nie-model4-spec4-q0
given_info     For husbands that don't set the alarm and wive...
question       Does husband positively affect alarm clock thr...
answer                                                        no
meta           {'story_id': 'alarm', 'graph_id': 'mediation',...
reasoning      {'step0': 'Let X = husband; V2 = wife; Y = ala...
step0               Let X = husband; V2 = wife; Y = alarm clock.
step1                                           X->V2,X->Y,V2->Y
step2                                    natural indirect effect
step3                           E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]
step4          P(Y=1 | X=0, V2=0) = 0.11\nP(Y=1 | X=0, V2=1) ...
step5          \sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X...
step6          0.01 * (0.60 - 0.11)+ 0.61 * (0.92 - 0.46)= -0.29
end                                                    -0.29 < 0
Name: 4, dtype: object

In [5]:
#create columnn with complete prompt to be given to the LLM
paraphrases = [
            "Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships. ",
            "Think of a self-contained, hypothetical setting with just the specified conditions, and devoid of any unknown factors or causal connections. ",
            "Consider a self-contained, hypothetical world with solely the mentioned conditions, and is free from any hidden factors or cause-and-effect relationships. ",
            "Imagine a self-contained, hypothetical setting with merely the stated conditions, and absent any unmentioned factors or causative links. ",
            "Think of a self-contained, hypothetical world with only the given conditions, and is void of any unknown factors or causative relationships. ",
        ]

prompt_end0 = " Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph that\
depicts the relationships in the scenario. "
prompt_end1 = "The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query \
type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explaining\
away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment\
effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed \
in quotation marks. Step 3) Extract all the available data. Your answer should contain nothing but marginal probabilities and conditional \
probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability being separated by a semicolon. Stick to the previously \
mentioned denotations for the variables. Based on all the reasoning above, output <YES> or <NO> to answer the following question: "

'''"The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine \
the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', \
'explaining away effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', \
'average treatment effect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the \
list above, enclosed in quotation marks. Step 3) Formalize the query: Translate the query into its formal mathematical expression based on \
its type, utilizing the 'do(·)' notation or counterfactual notations as needed. Step 4) Extract all the available data. Your answer should \
contain nothing but marginal probabilities and conditional probabilities in the form 'P(...)=...' or 'P(...|...)=...', each probability\
being separated by a semicolon. Stick to the previously mentioned denotations for the variables. Step 5) Given all the information above, \
deduce the estimand using skills such as do-calculus, counterfactual prediction, and the basics of probabilities. Answer step by step. \
Step 6) Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations, and derive the final answer. There \
is an identifiable answer. Answer step by step. \nBased on all the reasoning above, output one word to answer the initial question."'''


data['prompt'] = data.apply(lambda x: (paraphrases[0] + 'Question: ' + x['given_info']+prompt_end0+prompt_repres(x['reasoning'])+prompt_end1+x['question']),axis=1)

#df = data[['prompt', 'formal_form', 'graph', 'query_type']]
data

,question_id,desc_id,given_info,question,answer,meta,reasoning,step0,step1,step2,step3,step4,step5,step6,end,prompt
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",natural direct effect,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) ...","\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y...",0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = ...,0.32 > 0,"Imagine a self-contained, hypothetical world w..."
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",average treatment effect,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.26\nP(Y=1 | X=1) = 0.76,P(Y=1|X=1) - P(Y=1|X=0),0.76 - 0.26 = 0.50,0.50 > 0,"Imagine a self-contained, hypothetical world w..."
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",marginal probability,P(Y),P(X=1) = 0.77\nP(Y=1 | X=0) = 0.26\nP(Y=1 | X=...,P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0),0.77*0.76 - 0.23*0.26 = 0.64,0.64 > 0,"Imagine a self-contained, hypothetical world w..."
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",average treatment effect,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.20\nP(Y=1 | X=1) = 0.68,P(Y=1|X=1) - P(Y=1|X=0),0.68 - 0.20 = 0.49,0.49 > 0,"Imagine a self-contained, hypothetical world w..."
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",natural indirect effect,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.11\nP(Y=1 | X=0, V2=1) ...","\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X...",0.01 * (0.60 - 0.11)+ 0.61 * (0.92 - 0.46)= -0.29,-0.29 < 0,"Imagine a self-contained, hypothetical world w..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",normal counterfactual question,Y_{X=0} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 0 or 1,1,"Imagine a self-contained, hypothetical world w..."
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",normal counterfactual question,Y_{X=1} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 1 or 1,1,"Imagine a self-contained, hypothetical world w..."
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka 

In [6]:
df = pd.read_csv("/media/data/bazaluk/cladder_octopusv4.csv")
df

,question_id,4
0,desc_id,alarm-mediation-nde-model0-spec0-q0
1,given_info,For husbands that don't set the alarm and wive...
2,question,If we disregard the mediation effect through w...
3,answer,yes
4,meta,"{'story_id': 'alarm', 'graph_id': 'mediation',..."
5,reasoning,{'step0': 'Let X = husband; V2 = wife; Y = ala...
6,step0,Let X = husband; V2 = wife; Y = alarm clock.
7,step1,"X->V2,X->Y,V2->Y"
8,step2,natural direct effect
9,step3,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]"


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
torch.random.manual_seed(314159)

model = AutoModelForCausalLM.from_pretrained(
    "NexaAIDev/Octopus-v4", 
    device_map="cuda:0", 
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True 
)
tokenizer = AutoTokenizer.from_pretrained("NexaAIDev/Octopus-v4")

question = data['prompt'].iloc[0]

inputs = f"<|system|>You are a router. Below is the query from the users, please call the correct function and generate the parameters to call the function.<|end|><|user|>{question}<|end|><|assistant|>"
print('\n============= Below is the response ==============\n')

# You should consider to use early stopping with <nexa_end> token to accelerate
input_ids = tokenizer(inputs, return_tensors="pt")['input_ids'].to(model.device)

generated_token_ids = []
start = time.time()

# set a large enough number here to avoid insufficient length
for i in range(500):
    next_token = model(input_ids).logits[:, -1].argmax(-1)
    generated_token_ids.append(next_token.item())
    
    input_ids = torch.cat([input_ids, next_token.unsqueeze(1)], dim=-1)

    # 32041 is the token id of <nexa_end>
    if next_token.item() == 32041:
        break

print(tokenizer.decode(generated_token_ids))
#print(generated_token_ids)
end = time.time()
print(f'Elapsed time: {end - start:.2f}s')


/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
# Using open() function
file_path = "/media/data/bazaluk/cladder_octopusv4.csv"
	
print(f"File '{file_path}' created successfully.\n")

#generate answer for all the test data

query_type = ['marginal probability', 'conditional probability', 'explaining away effect', 'backdoor adjustment set', 
              'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatment effect on treated', 
              'natural direct effect', 'natural indirect effect']

data['predicted graph'] = None
data['predicted query type'] = None
data['predicted final answer'] = None
data['predicted text'] = None

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
torch.random.manual_seed(314159)

model = AutoModelForCausalLM.from_pretrained(
    "NexaAIDev/Octopus-v4", 
    device_map="cuda:0", 
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True 
)
tokenizer = AutoTokenizer.from_pretrained("NexaAIDev/Octopus-v4")

#for k in range(len(data)):
for k in range(2):
    question = data['prompt'].iloc[k]

    inputs = f"<|system|>You are a router. Below is the query from the users, please call the correct function and generate the parameters to call the function.<|end|><|user|>{question}<|end|><|assistant|>"

#print('\n============= Below is the response ==============\n')

# You should consider to use early stopping with <nexa_end> token to accelerate
    input_ids = tokenizer(inputs, return_tensors="pt")['input_ids'].to(model.device)

    generated_token_ids = []
#start = time.time()

# set a large enough number here to avoid insufficient length
    for i in range(500):
        next_token = model(input_ids).logits[:, -1].argmax(-1)
        generated_token_ids.append(next_token.item())
    
        input_ids = torch.cat([input_ids, next_token.unsqueeze(1)], dim=-1)

    # 32041 is the token id of <nexa_end>
        if next_token.item() == 32041:
            break

    text = tokenizer.decode(generated_token_ids)
    data.at[k, 'predicted text'] = text

    #check query_type
    for j in range(len(query_type)):
        if query_type[j] in text:
            data.at[k, 'predicted query type'] = query_type[j]

    #check graph
    import re

    if re.search('Step 1', text):
        step1_end = re.search('Step 1', text).end()
        if re.search('Step 2', text):
            step2_start = re.search('Step 2', text).start()
            data.at[k, 'predicted graph'] = text[step1_end:step2_start]
        else:
            data.at[k, 'predicted graph'] = None
    
    else:
        data.at[k, 'predicted graph'] = None
    
    #check final answer
    yes = len(re.findall('<yes>', text.lower()))
    no = len(re.findall('<no>', text.lower()))
    if yes >= no:
        data.at[k, 'predicted final answer'] = 'yes'
    else:
        data.at[k, 'predicted final answer'] = 'no'

    
    print(k)

data.to_csv(file_path, header=True, mode='a')
print("Finished.")
#print(generated_token_ids)
#end = time.time()
#print(f'Elapsed time: {end - start:.2f}s')
#data


File '/media/data/bazaluk/cladder_octopusv4.csv' created successfully.



`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0
1
Finished.


In [39]:
import re
a = "x -> v2, y -> v2 <YES>)"
#len(re.findall('<yes>', a.lower()))
re.search('\)', a).start()

22

In [2]:
pd.set_option('display.max_colwidth', None)

In [1]:
import pandas as pd
df = pd.read_csv('/media/data/bazaluk/cladder_octopusv4.csv')
#df.iloc[7]
#df_dist = df['prompt'][:10]
#df_dist
df

,Unnamed: 0,question_id,desc_id,given_info,question,answer,meta,reasoning,step0,step1,...,step3,step4,step5,step6,end,prompt,predicted graph,predicted query type,predicted final answer,predicted text
0,0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) ...","\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y...",0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = ...,0.32 > 0,"Imagine a self-contained, hypothetical world w...",: Analyze the causal relationships described i...,collider bias,yes,"""collider bias""\n\nStep 1: Analyze the causal ..."
1,1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.26\nP(Y=1 | X=1) = 0.76,P(Y=1|X=1) - P(Y=1|X=0),0.76 - 0.26 = 0.50,0.50 > 0,"Imagine a self-contained, hypothetical world w...",: Analyze the causal relationships between the...,conditional probability,yes,"""conditional probability"", ""marginal probabili..."
2,2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,P(Y),P(X=1) = 0.77\nP(Y=1 | X=0) = 0.26\nP(Y=1 | X=...,P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0),0.77*0.76 - 0.23*0.26 = 0.64,0.64 > 0,"Imagine a self-contained, hypothetical world w...",) Causal graph: X -> Y; X -> V2; X ~~ Y; X ~~ ...,conditional probability,yes,Step 1) Causal graph: X -> Y; X -> V2; X ~~ Y;...
3,3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.20\nP(Y=1 | X=1) = 0.68,P(Y=1|X=1) - P(Y=1|X=0),0.68 - 0.20 = 0.49,0.49 > 0,"Imagine a self-contained, hypothetical world w...",: Analyze the causal relationships between the...,conditional probability,yes,"""conditional probability"", ""marginal probabili..."
4,4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.11\nP(Y=1 | X=0, V2=1) ...","\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X...",0.01 * (0.60 - 0.11)+ 0.61 * (0.92 - 0.46)= -0.29,-0.29 < 0,"Imagine a self-contained, hypothetical world w...",NaN,collider bias,yes,"""collider bias""\n<|endoftext|><|endoftext|><|e..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",...,Y_{X=0} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 0 or 1,1,"Imagine a self-contained, hypothetical world w...",NaN,

In [7]:
for i in range(10):

    print('"'+df["prompt"].iloc[i]+'"')

    print(",")

"Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships. Question: For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%. Guidance: Address the question by following the steps below:
Step 1) Extract the causal graph: Identify the causal graph thatdepicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clo

In [10]:
#query acc
acc = 0
l = []

for i in range(len(df)):
    if df["step2"].iloc[i] == df["predicted query type"].iloc[i]:
        acc+=1
        if df["step2"].iloc[i] not in l:
            l.append(df["step2"].iloc[i])

acc = acc/len(df)
acc, l

(0.15916549460853258,
 ['natural indirect effect',
  'conditional probability',
  'marginal probability',
  'average treatment effect',
  'natural direct effect',
  'collider bias',
  'normal counterfactual question'])

In [12]:
#final answer acc
acc = 0
l = []

for i in range(len(df)):
    if df["answer"].iloc[i] == df["predicted final answer"].iloc[i]:
        acc+=1
        if df["step2"].iloc[i] not in l:
            l.append(df["step2"].iloc[i])

acc = acc/len(df)
acc, l

(0.5182841068917018,
 ['natural direct effect',
  'marginal probability',
  'natural indirect effect',
  'average treatment effect on treated',
  'average treatment effect',
  'conditional probability',
  'explaining away effect',
  'collider bias',
  'normal counterfactual question'])

In [6]:
def get_graph(text):
    for i in range(len(text)):
        if text[i] == '-' and text[i+1] == '->':
            break

    text = text[i:]

    return text.replace(' ','').replace('\n','').replace('.','').replace('\\n','')
#graph acc
acc = 0

for i in range(len(df)):
    if type(df['predicted graph'].iloc[i]) == str:
        #print(i)
        pred = get_graph(df['predicted graph'].iloc[i]).split(',')
        true_graph = df['step1'].iloc[i].split(',')

        #print(pred,true_graph)
        if set(pred) == set(true_graph):
            acc += 1

acc/len(df)

0.0

In [8]:
df['predicted graph'].iloc[2]

") Causal graph: X -> Y; X -> V2; X ~~ Y; X ~~ V2 ~~ Y. In this graph, X represents the husband, Y represents the alarm clock, and V2 represents the wife. The arrows indicate the causal relationships, with the first arrow showing the husband's influence on both the alarm and the wife, and the second arrow showing the husband's influence on the wife's interaction with the alarm. The double tilde (~~) represents a non-causal association between the variables.\n\n"

In [4]:
!pip3 install scikit-learn

In [3]:
from sklearn.metrics import f1_score
f1_score(df['answer'], df["predicted final answer"],average="weighted")

0.45643533139654024

In [13]:
data = df.copy()

#for i in range(len(df)):
#    data = df.dropna()
#    data["step2"].iloc[i] = a.index(data["step2"].iloc[i])
#    data["predicted query type"].iloc[i] = a.index(data["predicted query type"].iloc[i])

#from sklearn.metrics import f1_score
#f1_score(df['step2'], df["predicted query type"],average="weighted")

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [53]:
from sklearn.metrics import f1_score
f1_score(df['step2'], df["predicted query type"], average='weighted')

0.2591619404070644

In [48]:
import numpy as np
lista = list(df["predicted query type"].unique())
#np.append(lista,'normal counterfactual question')
lista.append("normal counterfactual question")

In [47]:
lista

In [49]:
dicio = dict(zip(lista,range(100)))

In [51]:
df["predicted query type"].replace(dicio)

/tmp/ipykernel_1056804/3180884369.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["predicted query type"].replace(dicio)


0       0
1       1
2       1
3       1
5       0
       ..
8519    1
8520    7
8521    1
8522    7
8523    7
Name: predicted query type, Length: 3691, dtype: int64